# Regression models

In deze notebook worden alle R-codes die gebruikt worden bij het uitvoeren van een sentimentanalyse op de canonenquête 2022 genoteerd. Deze zijn oorspronkelijk in RStudio verwerkt en zijn hierin geplaatst voor een duidelijkere uitleg. De oorspronkelijke R-code in .r-formaat zijn ook beschikbaar in deze GitHub.

## R-packages

Onderstaande blok code toont de verschillende packages die gebruikt zijn bij de sentimentanalyse.

- __lme4__: Gebruikt om de lineaire gemengde modellen te analyseren en te schatten
- __Matrix__: Gebruikt om de opslag en bewerkingen van grote datasets efficiënt te laten verlopen
- __effects__: Gebruikt om de effecten van de voorspellende factoren te visualiseren
- __dplyr__: Helpt bij het manipuleren en aanpassen van DataFrames, hierdoor was het mogelijk om te groeperen en te filteren
- __lmerTest__: Geeft de p-waarden en andere statistische testwaarden weer van de regressiemodellen
- __officer__: Gebruikt om Word documenten te maken in R
- __flextable__: Gebruikt om de samenvattingen om te zetten naar tabellen.


In [ ]:
library(lme4)
library(Matrix)
library(effects)
library(dplyr)
library(lmerTest)
library(officer)
library(flextable)

## Work directory

In onderstaande code werd de work directory bepaald en de dataset uit [1-tokenisation.ipynb](https://github.com/AntheSevenants/CanonComments/blob/main/1-tokenisation.ipynb) geladen.

In [ ]:
# set working directory + opening files
setwd("CanonComments")
LHT <- read.csv("sorted_data/Combined_Ratings.csv", header = TRUE,
                stringsAsFactors = TRUE)

## Filters en groeperingen

Bij sommigen modellen werden een aantal filters gebruikt om tegemoet te komen aan bepaalde tekorten in de dataset:

- __gender__: Gezien er niet genoeg data is om voor _x_ en _Zeg ik liever niet_ te analyseren werd er gefilterd op enkel man en vrouw. Deze filter wordt enkel toegepast bij de regressiemodellen waarin gender een variabele is.
- __afkomst__:
    - Gezien er niet genoeg data is om ieder land apart te analyseren werden deze gegroepeerd in 3 groepen. Deze groepering wordt enkel gebruikt wanneer afkomst een variabele is.
        - _De Lage Landen_: Nederland en België
        - _(Voormalige) Nederlandstalige gebieden buiten Europa_: Curaçao, Aruba, Suriname, Indonesië, Bonaire
        - _Andere_: de overige landen in de canonenquête
    - Daarnaast wordt er nog een model gebouwd waarbij het verschil tussen België en Nederland wordt geanalyseerd. Hiervoor wordt een filter gebruikt
- __opleiding__: Gezien het merendeel van de respondenten een universitair diploma heeft, wordt er groepering gebruikt om het verschil te analyseren tussen universitaire en niet-universitaire opleidingen. Dit wordt enkel gebruikt wanneer opleiding een variabele is in het model.

In [ ]:
# filter gender
LHT_filtered_gender <- LHT %>% filter(QPERS_GENDER %in% c("Man", "Vrouw"))

# groepering afkomst
LHT_filtered_birthcountry <- LHT %>%
  mutate(QPERS_BIRTH_COUNTRY_Grouped = case_when(
    QPERS_BIRTH_COUNTRY %in% c("België", "Nederland") ~ "De lage landen",
    QPERS_BIRTH_COUNTRY %in% c("Curaçao", "Aruba",
                               "Suriname", "Indonesië", "Bonaire")
    ~ "(Voormalige) Nederlandstalige gebieden buiten Europa",
    TRUE ~ "Andere" # Default case for all other countries
  ))
# filter België en Nederland
LHT_filtered_BENE <- LHT %>%
  filter(QPERS_BIRTH_COUNTRY %in% c("België", "Nederland"))

# groepering opleiding
LHT$QPERS_EDU_Grouped <-
  ifelse(LHT$QPERS_EDU == "Hoger onderwijs (universitair)",
         "universitair", "niet-universitair")

## Algemene modellen
Met algemene modellen worden de regressiemodellen aangeduid die alle data uit de commentaren van de canonenquête analyseren. Voor deze masterproef zijn er voor deze data 32 modellen gebouwd:
- __leeftijd__: 8 modellen
- __gender__: 8 modellen
- __opleidingsniveau__: 8 modellen
- __afkomst__: 8 modellen

De variabelen werden voor deze steeds handmatig aangepast en waarbij de samenvattingen handmatig geanalyseerd.

In [ ]:
# Het model wordt gebouwd op basis van de afhankelijke en
# onafhankelijke variabelen.
# eventueel wordt de gefilterde dataset geladen.
Age <- lmer(Surprise ~ QPERS_AGE + (1|response_id) + (1|questions), data = LHT)
summary(Age)
# Wanneer er signficante verbanden gevonden zijn, wordt er een effect gemaakt
# en gevisualiseerd in het oranje
eff <- effect("QPERS_AGE", Age)
plot(eff_data$QPERS_AGE, eff_data$fit, type = "l",
     col = "orange", lwd = 2,
     main = "Effect van leeftijd op valentie",
     xlab = "leeftijd", ylab = "valentie",
     xlim = range(eff_data$QPERS_AGE),
     ylim = range(c(eff_data$lower, eff_data$upper)))
# het betrouwbaarheidsinterval wordt toegevoegd
polygon(c(eff_data$QPERS_AGE, rev(eff_data$QPERS_AGE)),
        c(eff_data$lower, rev(eff_data$upper)),
        col = rgb(1, 0.5, 0, 0.2), border = NA)
# de rug (tick marks) worden toegevoegd
rug(eff_data$QPERS_AGE, col = "orange")
# het raster wordt toegevoegd zodat de grafiek leesbaarder is
grid(col = "grey", lty = "dotted")

## Onderwerpspecfieke modellen

Gezien er voor de onderwerpsspecifieke vragen steeds dezelfde formulie gebruikt kan worden, is er gebruikt gemaakt van _lapply_ om zeer snel 80 modellen te draaien. Hierdoor is het mogelijk om een soort van loop te creëren die de basis formule van een lineair regressiemodel (zie thesis voor reden) kan invullen met de afhankelijke variabelen en de juiste filter om vraag per vraag te behandelen.


In [ ]:
# opslaan van de variabelen die gebruikt worden in de modellen
dependent_vars <- c("Valence", "Arousal",
                    "Happiness", "Anger", "Fear", "Sadness",
                    "Disgust", "Surprise")
question_vars <- c("QI", "QC", "Q9", "Q10", "Q11",
                   "Q12_B", "Q14", "Q15", "Q16", "Q17")

# lapply loop die de vraag filteren uit de dataset
# + steeds iedere afhankelijke variabele invult
models2 <- lapply(question_vars, function(question) {
  data_subset <- LHT_filtered_gender[LHT_filtered_gender$questions ==
                                       question, ]
  lapply(dependent_vars, function(var) {
    formula <- as.formula(paste(var, "~ QPERS_AGE + QPERS_GENDER"))
    model <- lm(formula, data = data_subset)
    summary(model)
  })
})

# aan de hand van het cijfer tussen de twee
# is het mogelijk om de summaries van ieder
# model op te roepen
models2[[1]]

Onderstaande code werd dan gebruikt om het effect van een signficant verband te visualiseren. Dit is het voorbeeld voor vraag Q9 waar leeftijd en gender een significant effect hebben op geluk. Deze code is hergebruikt om dan andere visualisaties te maken.

In [ ]:
# De data wordt gefilterd op de specifieke vraag waar een verband is
data_subset <- subset(LHT_filtered_gender, questions == "Q9")
# Het model met leeftijd en gender wordt gemaakt
# om het gecombineerde effect te bekijken.
model <- lm(Happiness ~ QPERS_AGE + QPERS_GENDER, data = data_subset)
# ter controle wordt de samenvatting gecontroleerd met
# de samenvatting van lapply.
summary(model)

# Het effect van leeftijd wordt opnieuw berekend
# en geconverteerd naar een data.frame
eff_age <- effect("QPERS_AGE", model)
eff_age_df <- as.data.frame(eff_age)

# De dataset wordt gefilterd voor mannen
data_subset_male <- subset(data_subset, QPERS_GENDER == "Man")
# Het model wordt gebouwd dat het effect van leeftijd voor mannen nagaat
# en wordt berekend en opgeslagen in een data.frame
model_male <- lm(Happiness ~ QPERS_AGE, data = data_subset_male)
eff_age_male <- effect("QPERS_AGE", model_male)
eff_age_male_df <- as.data.frame(eff_age_male)

# Hetzelfde wordt gedaan voor het effect van leeftijd voor vrouwen
data_subset_female <- subset(data_subset, QPERS_GENDER == "Vrouw")
model_female <- lm(Happiness ~ QPERS_AGE, data = data_subset_female)
eff_age_female <- effect("QPERS_AGE", model_female)
eff_age_female_df <- as.data.frame(eff_age_female)

# Het effect voor mannen wordt gevisualiseerd in het blauw in de plot
plot(eff_age_male_df$QPERS_AGE, eff_age_male_df$fit, type = "l", col = "blue",
     main = "Effect van leeftijd en gender op opwinding voor Q9",
     xlab = "leeftijd", ylab = "opwinding",
     xlim = c(min(data_subset$QPERS_AGE), max(data_subset$QPERS_AGE)),
     ylim = range(c(eff_age_male_df$lower, eff_age_male_df$upper,
                    eff_age_female_df$lower, eff_age_female_df$upper)))

# Het 95%-betrouwbaarheidsinterval wordt berekend
polygon(c(eff_age_male_df$QPERS_AGE, rev(eff_age_male_df$QPERS_AGE)),
        c(eff_age_male_df$lower, rev(eff_age_male_df$upper)),
        col = rgb(0, 0, 1, 0.2), border = NA)

# De distributie voor mannen wordt toegevoegd als rug-plot
rug(data_subset_male$QPERS_AGE, col = "blue")

# Hetzelfde wordt in het geel gedaan voor vrouwen in de plot
lines(eff_age_female_df$QPERS_AGE, eff_age_female_df$fit, col = "yellow")

polygon(c(eff_age_female_df$QPERS_AGE, rev(eff_age_female_df$QPERS_AGE)),
        c(eff_age_female_df$lower, rev(eff_age_female_df$upper)),
        col = rgb(1, 1, 0, 0.2), border = NA)
rug(data_subset_female$QPERS_AGE, col = "yellow")

# de grid wordt toegevoegd zodat de grafiek leesbaarder is
grid()

# De legende wordt toegevoegd
legend("topleft", legend = c("mannen", "vrouwen"), col = c("blue", "yellow"),
       lty = 1, fill = c(rgb(0, 0, 1, 0.2), rgb(1, 1, 0, 0.2)))


## Export samenvattingen naar Word

Vanwege het grote aantal samenvattingen (112 modellen) werd er via [Copilot](https://copilot.microsoft.com/) een code gevraagd om deze samenvattingen eenvoudig te exporteren naar een Word document zodat dit niet manueel gedaan moest worden. Onderstaande code is dus niet zelf gecreëerd.

In [ ]:
# Functie om modelresumés om te zetten naar een flextable
# met afgeronde waarden en vertalingen
model_summary_to_flextable <- 
  function(model_summary, dependent_var, question_var) {
    coef_df <- as.data.frame(model_summary$coefficients)
    coef_df <- cbind(variabele = rownames(coef_df), coef_df)
    rownames(coef_df) <- NULL
    coef_df <- coef_df %>%
      mutate(across(where(is.numeric), ~ round(., 3))) %>%
      rename(schatting = Estimate, standaardfout = `Std. Error`,
             `t-waarde` = `t value`, `p-waarde` = `Pr(>|t|)`)
    coef_df$variabele <- ifelse(coef_df$variabele == "(Intercept)",
                                "constante (man)", coef_df$variabele)
    coef_df$variabele <- ifelse(coef_df$variabele == "QPERS_GENDERVrouw",
                                "verschil (vrouw - man)", coef_df$variabele)
    coef_df$variabele <- ifelse(coef_df$variabele == "QPERS_AGE",
                                "leeftijd", coef_df$variabele)
    coef_df <- mutate(coef_df, afhankelijke_variabele = dependent_var,
                      vraag = question_var)
    flextable(coef_df)
  }

# Creëer een nieuw Word-document
doc <- read_docx()

# Vertaal de namen van de afhankelijke variabelen naar het Nederlands
afhankelijke_variabelen <- c("Valence" = "valentie", "Arousal" = "opwinding",
                             "Happiness" = "geluk", "Anger" = "woede",
                             "Fear" = "angst", "Sadness" = "verdriet",
                             "Disgust" = "afgunst", "Surprise" = "verrassing")

# Converteer alle modelresumés naar flextables en voeg ze toe aan het document
for (i in seq_along(question_vars)) {
  for (j in seq_along(dependent_vars)) {
    model_summary <- models2[[i]][[j]]
    question_var <- question_vars[[i]]
    dependent_var <- dependent_vars[[j]]
    afhankelijke_variabele <- afhankelijke_variabelen[[dependent_var]]
    ft <- model_summary_to_flextable(model_summary,
                                     afhankelijke_variabele, question_var)
    # Voeg een titel toe vóór de tabel
    title <- paste(afhankelijke_variabele)
    doc <- body_add_par(doc, value = title, style = "Normal")
    # Voeg de flextable toe
    doc <- body_add_flextable(doc, value = ft)
    # Voeg een lege regel toe tussen de tabelle
    doc <- body_add_par(doc, value = "", style = "Normal")
  }
}

# Sla het Word-document op
print(doc, target = "model_summaries_dutch.docx")
